# **PyTorch Osnove 2: Razumevanje Modularnog Sistema PyTorch-a**

**Kurs:** KSMF1  
**Trajanje:** ~45 min  
**Preduslovi:** Osnove PyTorch tenzora (PyTorch Osnove 1)

---

## Ciljevi Sekcije

Do kraja ove sekcije, naučićete da:

- Razumete da je sve u PyTorch-u (slojevi, aktivacije, gubici) `nn.Module`
- Koristite **ugrađene** module kao što su `nn.Linear`, `nn.ReLU`, i `nn.MSELoss`
  - Izgradite jednostavne neuronske mreže kombinovanjem **ugrađenih** modula
- Kreirate **prilagođene** module od nule
  - Izgradite jednostavne neuronske mreže kombinovanjem **prilagođenih** modula
- Razumete konzistentan obrazac koji sve PyTorch komponente prate
---

# **Deo 3: Uvod u PyTorch Module**

## 3.1 Šta je Modul?

U PyTorch-u, **modul** je gradivni blok neuronske mreže (modela).

Posmatrajte PyTorch module kao **LEGO kockice** za gradnju neuronskih mreža. Baš kao što možete da kombinujete jednostavne LEGO delove da biste izgradili složene strukture, možete kombinovati jednostavne komponente neuronskih mreža da biste izgradili sofisticirane modele.

Ti gradivni blokovi su najčešće "koraci obrade" koji uzimaju neki ulaz x i vraćaju izlaz f(x). U ovom kontekstu, možemo videti da je "potpuno povezani sloj" modul, aktivaciona funkcija je takođe modul, čak su i funkcije gubitka moduli!

In [ ]:
# Učitajmo PyTorch i druge važne biblioteke
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# Postavi random seed za reproducibilnost
torch.manual_seed(42)
print("PyTorch version:", torch.__version__)

---

## 3.2 Osnovna klasa: **`nn.Module`**

Svaka komponenta neuronske mreže u PyTorch-u je podklasa (nasleđuje od) `nn.Module`. Ne brinite ako niste upoznati sa klasama - posmatrajte `nn.Module` kao **šablon** koji pruža zajedničku funkcionalnost za sve gradivne blokove neuronskih mreža.

U PyTorch-u, možete definisati svoje **prilagođene module**, ali on takođe dolazi sa velikim brojem već **ugrađenih modula** za lako i brzo implementiranje. Svi oni su **podklase** od `nn.Module`.

### Šta `nn.Module` Pruža

In [ ]:
# Hajde da pogledamo šta nn.Module pruža
print("Methods available in nn.Module:")
module_methods = [method for method in dir(nn.Module) if not method.startswith('_')]
print(f"Total methods: {len(module_methods)}")
print("List of methods:", module_methods)

### Najvažnije Metode

Ključne metode koje ćemo koristiti:

  - **forward():** Definiše kako podaci prolaze kroz modul (definiše **f(x; w)**)
  - **parameters():** Vraća sve parametre **w** koji se mogu trenirati
  - **train():** Postavlja modul u režim treniranja
  - **eval()**: Postavlja modul u režim evaluacije

---

## 3.3 Ugrađeni (Built-in) Moduli

PyTorch dolazi sa velikim brojem već definisanih modula za gradnju neuronskih mreža. Nazivamo ih **ugrađeni slojevi** i oni uključuju skoro sve vrste slojeva za obradu, aktivacionih slojeva i funkcija gubitka koje se često koriste.

Pogledajmo neke primere:

### Primer 1 - Linearni Slojevi (Potpuno Povezani Slojevi)

Linearni sloj izvršava operaciju: **izlaz = težine × ulaz + pomeraj**, gde je ulaz vektor veličine N, težine su MxN matrica, a pomeraj je vektor veličine M. Ovo čini izlazni vektor veličine M.

Ovo je bukvalno jednačina prave: **f(x) = wx + b**, ali proširena na više dimenzija (jednačina hiper-ravni)!

In [ ]:
# Kreirajmo linearni sloj koristeći ugrađenu klasu nn.Linear
# Ulazne karakteristike: 3 (npr., x, y, z koordinate)
# Izlazne karakteristike: 2 (npr., energija, impuls)

linear = nn.Linear(in_features=3, out_features=2)

# Ispitaj parametre
print(f"\nParameters:")
print(f"Weight shape: {linear.weight.shape}")  # (out_features, in_features)
print(f"Bias shape: {linear.bias.shape}")      # (out_features,)
print(f"Weight matrix:\n{linear.weight.data}")
print(f"Bias vector: {linear.bias.data}")

# Koristi sloj
input_data = torch.randn(4, 3)  # Kreiraj izmišljeni batch podataka od 4 uzorka, 3 karakteristike svaki
output = linear(input_data)  # Ovde koristimo sloj kao funkciju

print(f"\nUsage:")
print(f"Input shape: {input_data.shape}")
print(f"Output shape: {output.shape}")
print(f"Input:\n{input_data}")
print(f"Output:\n{output}")

# Proveri izračun ručno
manual_output = torch.matmul(input_data, linear.weight.t()) + linear.bias
print(f"\nManual computation matches PyTorch: {torch.allclose(output, manual_output)}")

### Primer 2 - Aktivacione Funkcije

Aktivacione funkcije uvode **nelinearnost** u neuronske mreže. Bez njih, višestruki linearni slojevi bi bili ekvivalentni jednom linearnom sloju!

Ispitajmo neke od najčešćih aktivacionih funkcija:

In [ ]:
# Pripremi x-osu za vizualizaciju
x = torch.linspace(-3, 3, 100)

# ReLU (Rectified Linear Unit): f(x) = max(0, x)
relu = nn.ReLU()
relu_output = relu(x)

# Sigmoid: f(x) = 1 / (1 + e^(-x))
sigmoid = nn.Sigmoid()
sigmoid_output = sigmoid(x)

# Tanh: f(x) = tanh(x)
tanh = nn.Tanh()
tanh_output = tanh(x)

# Nacrtaj aktivacione funkcije
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(x.numpy(), relu_output.numpy(), 'r-', linewidth=2)
plt.title('ReLU Activation')
plt.grid(True)
plt.xlabel('Input')
plt.ylabel('Output')

plt.subplot(1, 3, 2)
plt.plot(x.numpy(), sigmoid_output.numpy(), 'g-', linewidth=2)
plt.title('Sigmoid Activation')
plt.grid(True)
plt.xlabel('Input')
plt.ylabel('Output')

plt.subplot(1, 3, 3)
plt.plot(x.numpy(), tanh_output.numpy(), 'b-', linewidth=2)
plt.title('Tanh Activation')
plt.grid(True)
plt.xlabel('Input')
plt.ylabel('Output')

plt.tight_layout()
plt.show()

print("ReLU: Popularan jer je jednostavan i dobro radi")
print("Sigmoid: Izlazi između 0 i 1, dobar za verovatnoće")
print("Tanh: Izlazi između -1 i 1, centriran na nuli")

### Primer 3 - Funkcije Gubitka

Funkcije gubitka mere koliko su predviđanja modela pogrešna tako što uzimaju naš finalni izlaz i poznati rezultat iz dataset-a kao svoje ulaze, i primenjuju neku metriku da izmere koliko su različiti.

**Funkcije gubitka u PyTorch-u su takođe nn.Module-i!** To znači da prate iste obrasce koje smo učili.

In [ ]:
# Pogledajmo različite tipove funkcija gubitka
print("\n=== Common Loss Functions ===")

# Za regresiju (predviđanje kontinuiranih vrednosti)
print("Regression losses:")
print("- nn.MSELoss: Mean Squared Error")
print("- nn.L1Loss: Mean Absolute Error")
print("- nn.SmoothL1Loss: Huber loss (otporan na izuzetke)")

# Za klasifikaciju (predviđanje kategorija)
print("\nClassification losses:")
print("- nn.CrossEntropyLoss: Za multiklasnu klasifikaciju")
print("- nn.BCELoss: Binary Cross Entropy za binarnu klasifikaciju")
print("- nn.NLLLoss: Negative Log Likelihood")

# Kreiranje instanci
mse_loss = nn.MSELoss()
cross_entropy_loss = nn.CrossEntropyLoss()

Razumevanje Različitih Funkcija Gubitka

In [ ]:
# Uporedimo različite funkcije gubitka na primerima
print("=== Loss Function Comparison ===")

# Primer regresije
predicted_energy = torch.tensor([10.2, 15.8, 8.1])
actual_energy = torch.tensor([10.0, 16.0, 8.5])

mse = nn.MSELoss()(predicted_energy, actual_energy)
mae = nn.L1Loss()(predicted_energy, actual_energy)
smooth_l1 = nn.SmoothL1Loss()(predicted_energy, actual_energy)

print("Regression losses:")
print(f"Predictions: {predicted_energy}")
print(f"Actual: {actual_energy}")
print(f"MSE Loss: {mse:.4f}")
print(f"MAE Loss: {mae:.4f}")
print(f"Smooth L1 Loss: {smooth_l1:.4f}")

# Primer klasifikacije
raw_scores = torch.tensor([
    [2.1, 0.5, -1.2],  # Snažno predviđa klasu 0
    [-0.8, 1.9, 0.1],  # Snažno predviđa klasu 1
    [0.2, -1.1, 1.8]   # Snažno predviđa klasu 2
])
true_labels = torch.tensor([0, 1, 2])

cross_entropy = nn.CrossEntropyLoss()(raw_scores, true_labels)

print(f"\nClassification loss:")
print(f"Raw scores:\n{raw_scores}")
print(f"True labels: {true_labels}")
print(f"CrossEntropy Loss: {cross_entropy:.4f}")

---

## 3.4 Gradnja Neuronske Mreže sa Ugrađenim Modulima

Hajde da iskombinujemo ugrađene module da izgradimo jednostavnu neuronsku mrežu!

**VEOMA VAŽNO**: Baš kao što slojevi uzimaju ulaz x i vraćaju izlaz f(x), isto to radi i cela mreža. To znači da je **cela mreža zapravo `nn.Module`!**

Ovo nam omogućava da koristimo "slojeve apstrakcije", tj. da koristimo naše mreže kao gradivne blokove za još složenije strukture.

Postoje dva načina da se iskombinuju moduli u jedan veći modul (ili punu mrežu):
1. Korišćenje `nn.Sequential`: Ovo je modul koji automatski slaže module jedan za drugim. Pojednostavljuje gradnju mreža gde slojevi dolaze u sekvenci.
2. Definisanje većeg modula kao **prilagođeni modul**: Ovo je fleksibilniji metod za nesekvencijalno slaganje modula. Naučićemo kako da pravimo prilagođene module u sledećem odeljku.

### Korišćenje `nn.Sequential`

In [ ]:
# Metod 1: Korišćenje nn.Sequential (automatski slažemo naše slojeve)
simple_network = nn.Sequential(
    nn.Linear(4, 8),     # 4 ulazne karakteristike → 8 skrivenih jedinica
    nn.ReLU(),           # Aktivacijska funkcija
    nn.Linear(8, 4),     # 8 skrivenih jedinica → 4 skrivene jedinice
    nn.ReLU(),           # Još jedna aktivacija
    nn.Linear(4, 1)      # 4 skrivene jedinice → 1 izlaz
)

print("=== Simple Neural Network ===")
print("Network architecture:")
print(simple_network)
print(f"\nThe network is an nn.Module: {isinstance(simple_network, nn.Module)}")

# Testiraj mrežu
test_input = torch.randn(3, 4)  # 3 uzorka, 4 karakteristike svaki
network_output = simple_network(test_input)

print(f"\nTesting the network:")
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {network_output.shape}")
print(f"Input:\n{test_input}")
print(f"Output:\n{network_output}")

# Prebroj ukupne parametre
total_params = sum(p.numel() for p in simple_network.parameters())
print(f"\nTotal parameters: {total_params}")

# Detaljni pregled parametara
print("\nParameter breakdown:")
for i, layer in enumerate(simple_network):
    if hasattr(layer, 'weight'):
        weight_params = layer.weight.numel()
        bias_params = layer.bias.numel() if layer.bias is not None else 0
        total_layer_params = weight_params + bias_params
        print(f"Layer {i} ({layer.__class__.__name__}): {total_layer_params} parameters")
        print(f"  - Weights: {layer.weight.shape} = {weight_params} params")
        print(f"  - Bias: {layer.bias.shape} = {bias_params} params")
    else:
        print(f"Layer {i} ({layer.__class__.__name__}): 0 parameters")

### Testiranje sa Funkcijom Gubitka

In [ ]:
# Hajde da koristimo našu mrežu sa funkcijom gubitka
print("=== Network + Loss Function ===")

# Kreiraj neke lažne ciljne podatke za naš test
test_targets = torch.randn(3, 1)  # 3 uzorka, 1 cilj svaki
loss_function = nn.MSELoss()

# Izračunaj gubitak
loss = loss_function(network_output, test_targets)

print(f"Network predictions:\n{network_output}")
print(f"Target values:\n{test_targets}")
print(f"MSE Loss: {loss:.4f}")

---

## 3.5 Prilagođeni Moduli

Prava snaga PyTorch-a leži u **kreiranju vaših sopstvenih prilagođenih modula**. Da bismo razumeli kako ovo stvarno funkcioniše, ponovo ćemo kreirati neke od modula koje smo videli u prethodnom odeljku, ali ovog puta od nule.

U sledećim primerima, gradićemo `Linear layer`, `ReLU activation layer`, i `MSE loss function` kao potklase `nn.Modul`-a:

### Primer 1 - Prilagođeni Linearni Slojevi

Hajde da kreiramo `nn.Linear` sloj sami. Ovo će nam pokazati kako ugrađeni slojevi rade "ispod haube".

In [ ]:
class CustomLinear(nn.Module):
    """
    Naša sopstvena implementacija nn.Linear da bismo razumeli kako radi interno.
    Ovo je kao pravljenje sopstvenog voltmetra umesto kupovine već napravljenog -
    razumete tačno kako radi!
    """
    def __init__(self, in_features, out_features, bias=True):
        super(CustomLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        # Kreiraj matricu težina - ovo je ono što će mreža učiti!
        # BITNO: Moramo da je definišemo kao nn.Parameter da bi PyTorch znao da su to vrednosti koje treba da ažurira pri treningu
        self.weight = nn.Parameter(torch.randn(out_features, in_features))

        if bias:
            self.bias = nn.Parameter(torch.randn(out_features))
        else:
            self.bias = None

    def forward(self, x):
        """
        Forward pass: izračunaj output = input @ weight.T + bias
        Ovo je isto kao: output = weight @ input + bias za svaki uzorak
        """
        # Matrično množenje: (batch_size, in_features) @ (in_features, out_features)
        output = torch.matmul(x, self.weight.t())  # .t() znači transpozicija

        if self.bias is not None:
            output = output + self.bias

        return output

# Poredimo naš prilagođeni sloj sa PyTorch-ovim ugrađenim slojem

# Fiksiraj seed zbog reprudicibilnosti
torch.manual_seed(42)

# Kreiraj oba sloja sa istim dimenzijama
builtin_layer = nn.Linear(3, 2)
custom_layer = CustomLinear(3, 2)

# Kopiraj težine da budu identične za poređenje
custom_layer.weight.data = builtin_layer.weight.data.clone()
custom_layer.bias.data = builtin_layer.bias.data.clone()

# Testiraj na istim ulaznim podacima
test_input = torch.randn(4, 3)
builtin_output = builtin_layer(test_input)
custom_output = custom_layer(test_input)

print("Comparing built-in vs custom linear layer:")
print(f"Built-in output:\n{builtin_output}")
print(f"Custom output:\n{custom_output}")
print(f"Maximum difference: {torch.abs(builtin_output - custom_output).max():.10f}")
print("They're identical! Our custom layer works perfectly.")

### Primer 2 - Prilagođeni ReLU aktivacijski sloj

Baš kao i pre, hajde da izgradimo ReLU Aktivacioni Sloj od nule:

In [ ]:
class CustomReLU(nn.Module):
    """
    Naša sopstvena implementacija nn.ReLU.
    ReLU(x) = max(0, x)
    """
    def __init__(self):
        super(CustomReLU, self).__init__()
        # Aktivacione funkcije nemaju parametre za učenje!

    def forward(self, x):
        """Primeni ReLU aktivaciju: f(x) = max(0, x)"""
        return torch.clamp(x, min=0.0)  # Ekvivalentno sa torch.max(x, torch.zeros_like(x))

# Testiraj naš prilagođeni ReLU
print("=== Custom ReLU Activation ===")
custom_relu = CustomReLU()
builtin_relu = nn.ReLU()

print(f"Custom ReLU: {custom_relu}")
print(f"Built-in ReLU: {builtin_relu}")

# Testiraj sa pozitivnim i negativnim vrednostima
test_values = torch.tensor([-3.0, -1.0, 0.0, 1.0, 3.0])
custom_relu_output = custom_relu(test_values)
builtin_relu_output = builtin_relu(test_values)

print(f"\nTesting ReLU:")
print(f"Input: {test_values}")
print(f"Custom ReLU: {custom_relu_output}")
print(f"Built-in ReLU: {builtin_relu_output}")
print(f"Outputs are identical: {torch.allclose(custom_relu_output, builtin_relu_output)}")

# Proveri da nema parametre
print(f"\nCustom ReLU parameters: {len(list(custom_relu.parameters()))}")
print(f"Built-in ReLU parameters: {len(list(builtin_relu.parameters()))}")
print(f"Both have 0 parameters (activations don't learn!)")

# Vizuelno poređenje
x = torch.linspace(-3, 3, 100)
custom_y = custom_relu(x)
builtin_y = builtin_relu(x)

plt.figure(figsize=(8, 5))
plt.plot(x.numpy(), custom_y.numpy(), 'r-', linewidth=3, label='Custom ReLU', alpha=0.7)
plt.plot(x.numpy(), builtin_y.numpy(), 'b--', linewidth=2, label='Built-in ReLU')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title('Custom vs Built-in ReLU')
plt.legend()
plt.grid(True)
plt.show()

print("The curves are identical - our custom ReLU works perfectly!")

### Primer 3 - Prilagođena MSE Funkcija Gubitka

Kreirajmo našu sopstvenu MSE funkciju gubitka da bismo razumeli kako funkcije gubitka rade interno:

In [ ]:
class CustomMSELoss(nn.Module):
    """
    Naša sopstvena implementacija Mean Squared Error gubitka.
    MSE = mean((predicted - actual)²)
    """
    def __init__(self, reduction='mean'):
        super(CustomMSELoss, self).__init__()
        self.reduction = reduction

    def forward(self, predicted, actual):
        """Izračunaj MSE gubitak."""
        # Izračunaj kvadratne razlike
        squared_diff = (predicted - actual) ** 2

        # Primeni redukciju
        if self.reduction == 'mean':
            loss = torch.mean(squared_diff)
        elif self.reduction == 'sum':
            loss = torch.sum(squared_diff)
        elif self.reduction == 'none':
            loss = squared_diff
        else:
            raise ValueError(f"Invalid reduction: {self.reduction}")

        return loss

# Testiraj naš prilagođeni MSE
custom_mse = CustomMSELoss()
builtin_mse = nn.MSELoss()

# Test podaci
pred = torch.tensor([1.2, 2.8, 3.1])
actual = torch.tensor([1.0, 3.0, 3.5])

custom_loss = custom_mse(pred, actual)
builtin_loss = builtin_mse(pred, actual)

print("Comparing custom vs built-in MSE loss:")
print(f"Custom MSE: {custom_loss:.6f}")
print(f"Built-in MSE: {builtin_loss:.6f}")
print(f"Difference: {torch.abs(custom_loss - builtin_loss):.10f}")

# Ručno računanje za verifikaciju
manual_mse = torch.mean((pred - actual) ** 2)
print(f"Manual calculation: {manual_mse:.6f}")

---

## 3.6 Gradnja Neuronske Mreže sa Prilagođenim Modulima

Hajde sada da ponovo izgradimo našu neuronsku mrežu iz odeljka 3.4 koristeći samo prilagođene module koje smo napravili.

Ovaj put, umesto korišćenja nn.Sequential da brzo kombinujemo slojeve, kreiračemo našu sopstvenu klasu prilagođene mreže (zapamtite da su mreže takođe moduli, samo složeniji).

### Neuronska Mreža kao Prilagođeni `nn.Module`

In [ ]:
class CustomNetwork(nn.Module):
    """
    Prilagođena neuronska mreža izgrađena u potpunosti od naših prilagođenih modula.
    Ovo pokazuje kako se kreiraju složene mreže kombinovanjem jednostavnijih modula.
    """
    def __init__(self, input_size=4, hidden_size1=8, hidden_size2=4, output_size=1):
        super(CustomNetwork, self).__init__()

        # Definiši sve naše slojeve koristeći naše prilagođene module
        self.layer1 = CustomLinear(input_size, hidden_size1)
        self.activation1 = CustomReLU()
        self.layer2 = CustomLinear(hidden_size1, hidden_size2)
        self.activation2 = CustomReLU()
        self.layer3 = CustomLinear(hidden_size2, output_size)

        # Čuvaj informacije o arhitekturi
        self.architecture = f"{input_size}→{hidden_size1}→{hidden_size2}→{output_size}"

    def forward(self, x):
        """
        Definiši kako podaci prolaze kroz mrežu.
        Ovde specificiramo tačnu putanju izračunavanja.
        """
        # Forward pass kroz svaki sloj
        x = self.layer1(x)      # Linearna transformacija
        x = self.activation1(x)  # Primeni ReLU
        x = self.layer2(x)      # Još jedna linearna transformacija
        x = self.activation2(x)  # Još jedan ReLU
        x = self.layer3(x)      # Finalni linearni sloj (bez aktivacije)
        return x

    def get_info(self):
        """Vrati informacije o mreži."""
        total_params = sum(p.numel() for p in self.parameters())
        return f"CustomNetwork: {self.architecture}, Parameters: {total_params}"

# Izgradi našu prilagođenu mrežu
custom_network = CustomNetwork()
print("=== Custom Neural Network Class ===")
print(custom_network.get_info())
print(f"Network architecture:\n{custom_network}")

### Poređenje Prilagođene Mreže i `nn.Sequential` Mreže

In [ ]:
# Poredi sa nn.Sequential verzijom
sequential_network = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 4),
    nn.ReLU(),
    nn.Linear(4, 1)
)

print("\nSequential network architecture:")
print(sequential_network)

# Kopiraj težine da mreže budu identične za poređenje
custom_layers = [custom_network.layer1, custom_network.layer2, custom_network.layer3]
sequential_layers = [sequential_network[0], sequential_network[2], sequential_network[4]]

for custom_layer, seq_layer in zip(custom_layers, sequential_layers):
    custom_layer.weight.data = seq_layer.weight.data.clone()
    custom_layer.bias.data = seq_layer.bias.data.clone()

# Testiraj obe mreže sa istim ulazom
test_input = torch.randn(5, 4)
custom_output = custom_network(test_input)
sequential_output = sequential_network(test_input)

print(f"\nTesting both approaches:")
print(f"Input shape: {test_input.shape}")
print(f"Custom network output shape: {custom_output.shape}")
print(f"Sequential network output shape: {sequential_output.shape}")
print(f"Outputs are identical: {torch.allclose(custom_output, sequential_output)}")
print(f"Max difference: {torch.abs(custom_output - sequential_output).max():.10f}")

# Poređenje parametara
custom_params = sum(p.numel() for p in custom_network.parameters())
sequential_params = sum(p.numel() for p in sequential_network.parameters())

print(f"\nParameter count:")
print(f"Custom network: {custom_params}")
print(f"Sequential network: {sequential_params}")
print(f"Same parameter count: {custom_params == sequential_params}")

---

# **Deo 4: PyTorch Optimizatori**

## 4.1 Šta su Optimizatori?

U Delu 3, naučili smo kako da gradimo neuronske mreže koje mogu da prave predviđanja. Ali kako one zapravo **uče** iz podataka da naprave **tačna predviđanja**? E, tu dolaze optimizatori!

**Optimizator je kao trener** koji govori mreži kako da poboljša svoje performanse. Posmatrajte ovo na sledeći način:
- Vaša mreža pravi predviđanja (neka dobra, neka loša)
- Funkcija gubitka meri koliko su predviđanja pogrešna
- Optimizator koristi ovo merenje da shvati kako da prilagodi parametre mreže da pravi bolja predviđanja

U analogiji sa fizikom, zamislite da pokušavate da nađete najnižu tačku na potencijalnoj površi (minimum gubitka). Optimizator je vaša strategija za efikasno hodanje nizbrdo. Najčešće je to neka vrsta **gradijentnog spusta**.

In [ ]:
# Učitajmo PyTorch ponovo, ali ovaj put se uverimo da uključimo torch.optim kao optim
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# Postavi random seed za reproducibilnost
torch.manual_seed(42)
print("PyTorch version:", torch.__version__)

### Razumevanje Gradijentnog Spusta

In [ ]:
# Definiši našu jednostavnu 1D funkciju gubitka
def simple_loss_function(x):
    """Jednostavna kvadratna funkcija gubitka: f(x) = (x - 2)^2 + 1"""
    return (x - 2) ** 2 + 1

def gradient_of_loss(x):
    """Gradijent (izvod) naše funkcije gubitka: f'(x) = 2(x - 2)"""
    return 2 * (x - 2)

# Kreiraj tačke podataka za vizualizaciju
x_range = torch.linspace(-1, 5, 100)
loss_values = simple_loss_function(x_range)

# Početna tačka za naš "parametar"
x_current = torch.tensor(4.0, requires_grad=True)

# Kreiraj početnu vizualizaciju
plt.figure(figsize=(15, 5))

# Plot 1: Predeo gubitka (Loss landscape)
plt.subplot(1, 3, 1)
plt.plot(x_range.numpy(), loss_values.numpy(), 'b-', linewidth=2, label='Loss Function')
plt.axvline(x=2.0, color='r', linestyle='--', alpha=0.7, label='True Minimum (x=2)')

# Koristi .item() da dobiješ skalarnu vrednost za crtanje
plt.scatter([x_current.item()], [simple_loss_function(x_current).item()],
           color='orange', s=100, label=f'Starting Point (x={x_current.item():.1f})')
plt.xlabel('Parameter Value (x)')
plt.ylabel('Loss Value')
plt.title('Loss Landscape')
plt.legend()
plt.grid(True, alpha=0.3)

# Sada hajde da izvršimo gradijentni spust
learning_rate = 0.1
num_steps = 10
history = []

# Čuvaj početno stanje
with torch.no_grad():  # Koristi no_grad da izbegneš građenje computation graph-a
    history.append((x_current.item(), simple_loss_function(x_current).item()))

print("Gradient Descent Steps:")
print(f"Step 0: x = {x_current.item():.4f}, Loss = {simple_loss_function(x_current).item():.4f}")

for step in range(num_steps):
    # Izračunaj gubitak
    loss = simple_loss_function(x_current)

    # Izračunaj gradijent
    loss.backward()  # Ovo računa gradijent

    # Ažuriraj parametar koristeći gradijentni spust
    with torch.no_grad():
        x_current -= learning_rate * x_current.grad
        # Čuvaj istoriju za crtanje
        history.append((x_current.item(), simple_loss_function(x_current).item()))
        print(f"Step {step+1}: x = {x_current.item():.4f}, Loss = {simple_loss_function(x_current).item():.4f}, Gradient = {x_current.grad.item():.4f}")

    # Postavi gradijent na nulu za sledeću iteraciju
    x_current.grad.zero_()

# Plot 2: Progres optimizacije
plt.subplot(1, 3, 2)
steps = list(range(len(history)))
losses = [h[1] for h in history]
plt.plot(steps, losses, 'go-', linewidth=2, markersize=6)
plt.xlabel('Optimization Step')
plt.ylabel('Loss Value')
plt.title('Loss During Optimization')
plt.grid(True, alpha=0.3)

# Plot 3: Putanja parametra na predelu gubitka
plt.subplot(1, 3, 3)
plt.plot(x_range.numpy(), loss_values.numpy(), 'b-', linewidth=2, alpha=0.7, label='Loss Function')
plt.axvline(x=2.0, color='r', linestyle='--', alpha=0.7, label='True Minimum')

# Nacrtaj putanju koju je prošao gradijentni spust
x_positions = [h[0] for h in history]
y_positions = [h[1] for h in history]
plt.plot(x_positions, y_positions, 'go-', linewidth=2, markersize=6, alpha=0.8, label='GD Path')
plt.scatter([x_positions[0]], [y_positions[0]], color='orange', s=100, label='Start', zorder=5)
plt.scatter([x_positions[-1]], [y_positions[-1]], color='red', s=100, label='End', zorder=5)

plt.xlabel('Parameter Value (x)')
plt.ylabel('Loss Value')
plt.title('Gradient Descent Path')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Rezime
print(f"\nSummary:")
print(f"Started at x = {history[0][0]:.4f} with loss = {history[0][1]:.4f}")
print(f"Ended at x = {history[-1][0]:.4f} with loss = {history[-1][1]:.4f}")
print(f"True minimum is at x = 2.0000 with loss = 1.0000")
print(f"Error from true minimum: {abs(history[-1][0] - 2.0):.4f}")

### Vizuelizacija Efekata Stope Učenja

In [ ]:
# Simuliraj gradijentni spust sa različitim stopama učenja
def gradient_descent_simulation(learning_rate, num_steps=15):
    """Simuliraj gradijentni spust na našoj jednostavnoj funkciji"""
    x = 4  # Početna tačka
    history = [x]
    loss_history = [simple_loss_function(torch.tensor(x)).item()]

    for step in range(num_steps):
        gradient = gradient_of_loss(torch.tensor(x))
        x = x - learning_rate * gradient.item()
        history.append(x)
        loss_history.append(simple_loss_function(torch.tensor(x)).item())

    return history, loss_history

# Testiraj različite stope učenja
learning_rates = [0.1, 0.3, 1.0, 1.03]
colors = ['blue', 'green', 'red', 'purple']

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
# Nacrtaj funkciju gubitka
x_range = torch.linspace(-1, 5, 100)
loss_values = simple_loss_function(x_range)
plt.plot(x_range.numpy(), loss_values.numpy(), 'k-', linewidth=2, alpha=0.3, label='Loss Function')

for lr, color in zip(learning_rates, colors):
    x_hist, loss_hist = gradient_descent_simulation(lr)
    # Nacrtaj putanju na funkciji gubitka
    x_points = torch.tensor(x_hist)
    y_points = simple_loss_function(x_points)
    plt.plot(x_points.numpy(), y_points.numpy(), 'o-', color=color, alpha=0.7,
            markersize=4, label=f'LR = {lr}')

plt.axvline(x=2.0, color='r', linestyle='--', alpha=0.5, label='True Minimum')
plt.xlabel('Parameter Value')
plt.ylabel('Loss')
plt.title('Gradient Descent Paths')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
for lr, color in zip(learning_rates, colors):
    x_hist, loss_hist = gradient_descent_simulation(lr)
    plt.plot(loss_hist, 'o-', color=color, alpha=0.7, markersize=4, label=f'LR = {lr}')

plt.xlabel('Optimization Step')
plt.ylabel('Loss Value')
plt.title('Loss vs Training Steps')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Efekti različitih stopa učenja:")
print("🔵 LR = 0.1: Spor ali stabilan napredak")
print("🟢 LR = 0.5: Dobra ravnoteža brzine i stabilnosti")
print("🔴 LR = 1.0: Na granici stabilnosti")
print("🟣 LR = 1.03: Previsoka! Oscilacije i nestabilnost")

---

## 4.2 Najčešće Korišćeni Optimizatori

Da biste pristupili ugrađenim PyTorch optimizatorima, ne zaboravite da uradite `import torch.optim as optim`. Ako želite, možete takođe napraviti prilagođeni optimizator, ali to nećemo raditi na ovom kursu.

Pogledajmo najpopularnije optimizatore u PyTorch-u:
*   SGD (Stochastic Gradient Descent)
*   Adam (Adaptive Moment Estimation) - Veoma popularan!
*   RMSprop

In [ ]:
# Kreiraj jednostavan model za demonstraciju optimizatora
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

# Kreiraj model i neke lažne podatke
model = SimpleModel()
x_data = torch.randn(10, 1)
y_data = torch.randn(10, 1)

print("=== Common Optimizers ===")

# 1. SGD (Stochastic Gradient Descent)
sgd_optimizer = optim.SGD(model.parameters(), lr=0.01)
print(f"SGD: {sgd_optimizer}")

# 2. Adam (Adaptive Moment Estimation)
adam_optimizer = optim.Adam(model.parameters(), lr=0.01)
print(f"Adam: {adam_optimizer}")

# 3. RMSprop
rmsprop_optimizer = optim.RMSprop(model.parameters(), lr=0.01)
print(f"RMSprop: {rmsprop_optimizer}")

print(f"\nKey parameters:")
print(f"- lr (learning rate): Koliko veliki koraci da se prave")
print(f"- Većina optimizatora ima dodatne parametre za fino podešavanje")

---

# Ključni Zaključci

## Šta smo naučili:

1. **Univerzalni obrazac**: Gradivni blokovi u PyTorch-u (slojevi, aktivacije, gubici) su podklase osnovne `nn.Module` klase
2. **Ugrađeni moduli**: Već napravljene komponente koje prate konzistentne interfejse
3. **Prilagođeni moduli**: Kako da izgradite svoje komponente od nule
4. **Kompozicija modula**: Kako da kombinujete module u veće sisteme
5. **Ispod haube**: Šta se stvarno dešava unutar čestih PyTorch komponenti
6. **Optimizatori**: Algoritmi koji čine da neuronske mreže uče

## PyTorch filozofija:

- **Modularnost**: Mali, iznova upotrebljivi delovi
- **Konzistentnost**: Isti interfejs za sve komponente
- **Transparentnost**: Lako je videti i menjati šta se dešava unutra
- **Kompozibilnost**: Jednostavni delovi se kombinuju da naprave složene sisteme

## Sledeći koraci:

U sledećem odeljku, naučićemo o **Radnim tokovima u PyTorch-u** - kako da stvarno sve spojimo u jednu celinu. Koristićemo module koje sada razumemo i dodati finalne delove potrebne za mašinsko učenje.

---

# Samostalni Rad

Zapamtite, svaki prilagođeni modul treba da ima:
1. super().**__init__**() u **init**
2. nn.Parameter() ako ima parametre koji se mogu učiti
3. forward() metodu za izračunavanje

## Zadaci

1. Kreirajte prilagođenu Sigmoid aktivaciju
    - Savet: sigmoid(x) = 1 / (1 + exp(-x))
    - Koristite torch.exp() i pazite na numeričku stabilnost
2. Kreirajte prilagođenu L1Loss (Mean Absolute Error) funkciju gubitka
    - Savet: L1(prediction, target) = mean(|prediction - target|)
    - Koristite torch.abs()
3. Kreirajte prilagođeni sloj koji dodaje "učljiv" pomeraj  **b** bilo kom ulazu **f(x) = x + b**
    - Ovaj sloj treba da doda različit pomeraj svakoj karakteristici ulaznih podataka
4. Kombinujte svoje prilagođene module
    - Izgradite mrežu koristeći vašu prilagođenu Sigmoid i L1Loss
    - Izgradite je sa PyTorch-ovim ugrađenim ekvivalentima

In [ ]:
# Mesto za rešavanje Zadatka 1

In [ ]:
# Mesto za rešavanje Zadatka 2

In [ ]:
# Mesto za rešavanje Zadatka 3

In [ ]:
# Mesto za rešavanje Zadatka 4

---

## Rešenja

In [ ]:
# -------- Zadatak 1 --------
import torch
import torch.nn as nn

class CustomSigmoid(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        # Numerički stabilna sigmoid funkcija:
        # Za x >= 0: sigmoid(x) = 1 / (1 + exp(-x))
        # Za x < 0 : sigmoid(x) = exp(x) / (1 + exp(x))
        out = torch.empty_like(x)
        pos_mask = (x >= 0)
        neg_mask = ~pos_mask

        # Stabilno izračunavanje za pozitivne vrednosti
        out[pos_mask] = 1 / (1 + torch.exp(-x[pos_mask]))

        # Stabilno izračunavanje za negativne vrednosti
        exp_x = torch.exp(x[neg_mask])
        out[neg_mask] = exp_x / (1 + exp_x)

        return out

In [ ]:
# -------- Zadatak 2 --------
import torch
import torch.nn as nn

class CustomL1Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred, target):
        return torch.mean(torch.abs(pred - target))


In [ ]:
# -------- Zadatak 3 --------
import torch
import torch.nn as nn

class BiasLayer(nn.Module):
    """
    Dodaje učljiv pomeraj svakoj karakteristici: f(x) = x + b
    Radi za ulaze oblika (batch_size, num_features).
    """
    def __init__(self, num_features):
        super().__init__()
        # bias (pomeraj) je učljiv parametar oblika (num_features,)
        self.bias = nn.Parameter(torch.zeros(num_features))

    def forward(self, x):
        # Broadcasting automatski dodaje bias svakom uzorku u batch-u
        return x + self.bias

In [ ]:
# -------- Zadatak 4 --------
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)
N, D_in, D_out = 512, 10, 1

# ---- Dva modela: (A) prilagođen, (B) ugrađen ----
class TinyNetCustom(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(D_in, D_out, bias=False)
        self.bias = BiasLayer(D_out)        # učljiv bias
        self.act = CustomSigmoid()          # prilagođena sigmoid funkcija

    def forward(self, x):
        return self.act(self.bias(self.lin(x)))

class TinyNetBuiltin(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(D_in, D_out, bias=False)
        self.bias = BiasLayer(D_out)

    def forward(self, x):
        return torch.sigmoid(self.bias(self.lin(x)))

netA = TinyNetCustom()
netB = TinyNetBuiltin()

# Inicijalizuj obe mreže sa istim težinama za ravnopravno poređenje
with torch.no_grad():
    netB.lin.weight.copy_(netA.lin.weight)
    netB.bias.bias.copy_(netA.bias.bias)

lossA = CustomL1Loss()
lossB = nn.L1Loss()

optA = optim.SGD(netA.parameters(), lr=0.5)
optB = optim.SGD(netB.parameters(), lr=0.5)